<a href="https://colab.research.google.com/github/VinishUchiha/GenerativeModelling/blob/master/StackGANstage1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz

--2020-05-31 08:39:11--  http://www.vision.caltech.edu/visipedia-data/CUB-200-2011/CUB_200_2011.tgz
Resolving www.vision.caltech.edu (www.vision.caltech.edu)... 34.208.54.77
Connecting to www.vision.caltech.edu (www.vision.caltech.edu)|34.208.54.77|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1150585339 (1.1G) [application/x-tar]
Saving to: ‘CUB_200_2011.tgz’

CUB_200_2011.tgz    100%[===================>]   1.07G   818KB/s    in 34m 19s 

2020-05-31 09:13:31 (546 KB/s) - ‘CUB_200_2011.tgz’ saved [1150585339/1150585339]



In [2]:
!tar -xvzf CUB_200_2011.tgz

Streaming output truncated to the last 5000 lines.
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0079_113288.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0004_111989.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0068_113247.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0127_111935.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0094_112773.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0028_113217.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0126_110959.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0007_111029.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0096_111519.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0120_113001.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0010_112678.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0025_111669.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0084_111300.jpg
CUB_200_2011/images/118.House_Sparrow/House_Sparrow_0002_112905.j

In [3]:
!unzip /content/drive/My\ Drive/birds.zip

Streaming output truncated to the last 5000 lines.
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0036_797287.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0063_27123.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0046_797295.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0072_26993.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0042_27143.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0029_797300.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0031_797299.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0034_797305.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Rosy_Finch_0027_27153.txt  
  inflating: birds/text_c10/034.Gray_crowned_Rosy_Finch/Gray_Crowned_Ro

In [0]:
!cp /content/CUB_200_2011.tgz /content/drive/My\ Drive

In [0]:
!cp -R /content/CUB_200_2011 /content/birds

In [0]:
import os
import pickle
import random
import time

import PIL
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
from tensorflow.keras import Input, Model
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization, ReLU, Reshape, UpSampling2D, Conv2D, Activation, \
    concatenate, Flatten, Lambda, Concatenate
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt

In [0]:
def load_class_ids(class_info_file_path):
    """
    Load class ids from class_info.pickle file
    """
    with open(class_info_file_path, 'rb') as f:
        class_ids = pickle.load(f, encoding='latin1')
        return class_ids

In [0]:
def load_embeddings(embeddings_file_path):
    """
    Load embeddings
    """
    with open(embeddings_file_path, 'rb') as f:
        embeddings = pickle.load(f, encoding='latin1')
        embeddings = np.array(embeddings)
        print('embeddings: ', embeddings.shape)
    return embeddings

In [0]:
def load_filenames(filenames_file_path):
    """
    Load filenames.pickle file and return a list of all file names
    """
    with open(filenames_file_path, 'rb') as f:
        filenames = pickle.load(f, encoding='latin1')
    return filenames

In [0]:
def load_bounding_boxes(dataset_dir):
    """
    Load bounding boxes and return a dictionary of file names and corresponding bounding boxes
    """
    # Paths
    bounding_boxes_path = os.path.join(dataset_dir, 'bounding_boxes.txt')
    file_paths_path = os.path.join(dataset_dir, 'images.txt')

    # Read bounding_boxes.txt and images.txt file
    df_bounding_boxes = pd.read_csv(bounding_boxes_path,
                                    delim_whitespace=True, header=None).astype(int)
    df_file_names = pd.read_csv(file_paths_path, delim_whitespace=True, header=None)

    # Create a list of file names
    file_names = df_file_names[1].tolist()

    # Create a dictionary of file_names and bounding boxes
    filename_boundingbox_dict = {img_file[:-4]: [] for img_file in file_names[:2]}

    # Assign a bounding box to the corresponding image
    for i in range(0, len(file_names)):
        # Get the bounding box
        bounding_box = df_bounding_boxes.iloc[i][1:].tolist()
        key = file_names[i][:-4]
        filename_boundingbox_dict[key] = bounding_box

    return filename_boundingbox_dict

In [0]:
def get_img(img_path, bbox, image_size):
    """
    Load and resize image
    """
    img = Image.open(img_path).convert('RGB')
    width, height = img.size
    if bbox is not None:
        R = int(np.maximum(bbox[2], bbox[3]) * 0.75)
        center_x = int((2 * bbox[0] + bbox[2]) / 2)
        center_y = int((2 * bbox[1] + bbox[3]) / 2)
        y1 = np.maximum(0, center_y - R)
        y2 = np.minimum(height, center_y + R)
        x1 = np.maximum(0, center_x - R)
        x2 = np.minimum(width, center_x + R)
        img = img.crop([x1, y1, x2, y2])
    img = img.resize(image_size, PIL.Image.BILINEAR)
    return img

In [0]:
def load_dataset(filenames_file_path, class_info_file_path, cub_dataset_dir, embeddings_file_path, image_size):
    """
    Load dataset
    """
    filenames = load_filenames(filenames_file_path)
    class_ids = load_class_ids(class_info_file_path)
    bounding_boxes = load_bounding_boxes(cub_dataset_dir)
    all_embeddings = load_embeddings(embeddings_file_path)

    X, y, embeddings = [], [], []

    print("Embeddings shape:", all_embeddings.shape)

    for index, filename in enumerate(filenames):
        bounding_box = bounding_boxes[filename]

        try:
            # Load images
            img_name = '{}/images/{}.jpg'.format(cub_dataset_dir, filename)
            img = get_img(img_name, bounding_box, image_size)

            all_embeddings1 = all_embeddings[index, :, :]

            embedding_ix = random.randint(0, all_embeddings1.shape[0] - 1)
            embedding = all_embeddings1[embedding_ix, :]

            X.append(np.array(img))
            y.append(class_ids[index])
            embeddings.append(embedding)
        except Exception as e:
            print(e)

    X = np.array(X)
    y = np.array(y)
    embeddings = np.array(embeddings)
    return X, y, embeddings


In [0]:
def generate_c(x):
    mean = x[:, :128]
    log_sigma = x[:, 128:]
    stddev = K.exp(log_sigma)
    epsilon = K.random_normal(shape=K.constant((mean.shape[1],), dtype='int32'))
    c = stddev * epsilon + mean
    return c

In [0]:
def build_ca_model():
    """
    Get conditioning augmentation model.
    Takes an embedding of shape (1024,) and returns a tensor of shape (256,)
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)
    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_embedding_compressor_model():
    """
    Build embedding compressor model
    """
    input_layer = Input(shape=(1024,))
    x = Dense(128)(input_layer)
    x = ReLU()(x)

    model = Model(inputs=[input_layer], outputs=[x])
    return model

In [0]:
def build_stage1_generator():
    """
    Builds a generator model used in Stage-I
    """
    input_layer = Input(shape=(1024,))
    x = Dense(256)(input_layer)
    mean_logsigma = LeakyReLU(alpha=0.2)(x)

    c = Lambda(generate_c)(mean_logsigma)

    input_layer2 = Input(shape=(100,))

    gen_input = Concatenate(axis=1)([c, input_layer2])

    x = Dense(128 * 8 * 4 * 4, use_bias=False)(gen_input)
    x = ReLU()(x)

    x = Reshape((4, 4, 128 * 8), input_shape=(128 * 8 * 4 * 4,))(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(512, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(256, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(128, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = UpSampling2D(size=(2, 2))(x)
    x = Conv2D(64, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(3, kernel_size=3, padding="same", strides=1, use_bias=False)(x)
    x = Activation(activation='tanh')(x)

    stage1_gen = Model(inputs=[input_layer, input_layer2], outputs=[x, mean_logsigma])
    return stage1_gen


In [0]:

def build_stage1_discriminator():
    """
    Create a model which takes two inputs
    1. One from above network
    2. One from the embedding layer
    3. Concatenate along the axis dimension and feed it to the last module which produces final logits
    """
    input_layer = Input(shape=(64, 64, 3))

    x = Conv2D(64, (4, 4),
               padding='same', strides=2,
               input_shape=(64, 64, 3), use_bias=False)(input_layer)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(512, (4, 4), padding='same', strides=2, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    input_layer2 = Input(shape=(4, 4, 128))

    merged_input = concatenate([x, input_layer2])

    x2 = Conv2D(64 * 8, kernel_size=1,
                padding="same", strides=1)(merged_input)
    x2 = BatchNormalization()(x2)
    x2 = LeakyReLU(alpha=0.2)(x2)
    x2 = Flatten()(x2)
    x2 = Dense(1)(x2)
    x2 = Activation('sigmoid')(x2)

    stage1_dis = Model(inputs=[input_layer, input_layer2], outputs=[x2])
    return stage1_dis

In [0]:
def build_adversarial_model(gen_model, dis_model):
    input_layer = Input(shape=(1024,))
    input_layer2 = Input(shape=(100,))
    input_layer3 = Input(shape=(4, 4, 128))

    x, mean_logsigma = gen_model([input_layer, input_layer2])

    dis_model.trainable = False
    valid = dis_model([x, input_layer3])

    model = Model(inputs=[input_layer, input_layer2, input_layer3], outputs=[valid, mean_logsigma])
    return model

In [0]:
def KL_loss(y_true, y_pred):
    mean = y_pred[:, :128]
    logsigma = y_pred[:, :128]
    loss = -logsigma + .5 * (-1 + K.exp(2. * logsigma) + K.square(mean))
    loss = K.mean(loss)
    return loss

In [0]:
def custom_generator_loss(y_true, y_pred):
    # Calculate binary cross entropy loss
    return K.binary_crossentropy(y_true, y_pred)

In [0]:
def save_rgb_img(img, path):
    """
    Save an rgb image
    """
    fig = plt.figure()
    ax = fig.add_subplot(1, 1, 1)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title("Image")

    plt.savefig(path)
    plt.close()

In [0]:
data_dir = "birds/"
train_dir = data_dir + "/train"
test_dir = data_dir + "/test"
image_size = 64
batch_size = 64
z_dim = 100
stage1_generator_lr = 0.0002
stage1_discriminator_lr = 0.0002
stage1_lr_decay_step = 600
epochs = 1000
condition_dim = 128

embeddings_file_path_train = train_dir + "/char-CNN-RNN-embeddings.pickle"
embeddings_file_path_test = test_dir + "/char-CNN-RNN-embeddings.pickle"

filenames_file_path_train = train_dir + "/filenames.pickle"
filenames_file_path_test = test_dir + "/filenames.pickle"

class_info_file_path_train = train_dir + "/class_info.pickle"
class_info_file_path_test = test_dir + "/class_info.pickle"

cub_dataset_dir = data_dir + "/CUB_200_2011"

# Define optimizers
dis_optimizer = Adam(lr=stage1_discriminator_lr, beta_1=0.5, beta_2=0.999)
gen_optimizer = Adam(lr=stage1_generator_lr, beta_1=0.5, beta_2=0.999)

""""
Load datasets
"""
X_train, y_train, embeddings_train = load_dataset(filenames_file_path=filenames_file_path_train,
                                                  class_info_file_path=class_info_file_path_train,
                                                  cub_dataset_dir=cub_dataset_dir,
                                                  embeddings_file_path=embeddings_file_path_train,
                                                  image_size=(64, 64))

X_test, y_test, embeddings_test = load_dataset(filenames_file_path=filenames_file_path_test,
                                                class_info_file_path=class_info_file_path_test,
                                                cub_dataset_dir=cub_dataset_dir,
                                                embeddings_file_path=embeddings_file_path_test,
                                                image_size=(64, 64))



In [27]:
"""
Build and compile networks
"""
ca_model = build_ca_model()
ca_model.compile(loss="binary_crossentropy", optimizer="adam")

stage1_dis = build_stage1_discriminator()
stage1_dis.compile(loss='binary_crossentropy', optimizer=dis_optimizer)

stage1_gen = build_stage1_generator()
stage1_gen.compile(loss="mse", optimizer=gen_optimizer)

embedding_compressor_model = build_embedding_compressor_model()
embedding_compressor_model.compile(loss="binary_crossentropy", optimizer="adam")

adversarial_model = build_adversarial_model(gen_model=stage1_gen, dis_model=stage1_dis)
adversarial_model.compile(loss=['binary_crossentropy', KL_loss], loss_weights=[1, 2.0],
                          optimizer=gen_optimizer, metrics=None)

tensorboard = TensorBoard(log_dir="logs/".format(time.time()))
tensorboard.set_model(stage1_gen)
tensorboard.set_model(stage1_dis)
tensorboard.set_model(ca_model)
tensorboard.set_model(embedding_compressor_model)

# Generate an array containing real and fake values
# Apply label smoothing as well
real_labels = np.ones((batch_size, 1), dtype=float) * 0.9
fake_labels = np.zeros((batch_size, 1), dtype=float) * 0.1

for epoch in range(epochs):
    print("========================================")
    print("Epoch is:", epoch)
    print("Number of batches", int(X_train.shape[0] / batch_size))

    gen_losses = []
    dis_losses = []

    # Load data and train model
    number_of_batches = int(X_train.shape[0] / batch_size)
    for index in range(number_of_batches):
        print("Batch:{}".format(index+1))
        
        """
        Train the discriminator network
        """
        # Sample a batch of data
        z_noise = np.random.normal(0, 1, size=(batch_size, z_dim))
        image_batch = X_train[index * batch_size:(index + 1) * batch_size]
        embedding_batch = embeddings_train[index * batch_size:(index + 1) * batch_size]
        image_batch = (image_batch - 127.5) / 127.5

        # Generate fake images
        fake_images, _ = stage1_gen.predict([embedding_batch, z_noise], verbose=3)

        # Generate compressed embeddings
        compressed_embedding = embedding_compressor_model.predict_on_batch(embedding_batch)
        compressed_embedding = np.reshape(compressed_embedding, (-1, 1, 1, condition_dim))
        compressed_embedding = np.tile(compressed_embedding, (1, 4, 4, 1))

        dis_loss_real = stage1_dis.train_on_batch([image_batch, compressed_embedding],
                                                  np.reshape(real_labels, (batch_size, 1)))
        dis_loss_fake = stage1_dis.train_on_batch([fake_images, compressed_embedding],
                                                  np.reshape(fake_labels, (batch_size, 1)))
        dis_loss_wrong = stage1_dis.train_on_batch([image_batch[:(batch_size - 1)], compressed_embedding[1:]],
                                                    np.reshape(fake_labels[1:], (batch_size-1, 1)))

        d_loss = 0.5 * np.add(dis_loss_real, 0.5 * np.add(dis_loss_wrong, dis_loss_fake))

        print("d_loss_real:{}".format(dis_loss_real))
        print("d_loss_fake:{}".format(dis_loss_fake))
        print("d_loss_wrong:{}".format(dis_loss_wrong))
        print("d_loss:{}".format(d_loss))

        """
        Train the generator network 
        """
        g_loss = adversarial_model.train_on_batch([embedding_batch, z_noise, compressed_embedding],[K.ones((batch_size, 1)) * 0.9, K.ones((batch_size, 256)) * 0.9])
        print("g_loss:{}".format(g_loss))

        dis_losses.append(d_loss)
        gen_losses.append(g_loss)
    
    # Generate and save images after every 2nd epoch
    if epoch % 2 == 0:
        # z_noise2 = np.random.uniform(-1, 1, size=(batch_size, z_dim))
        z_noise2 = np.random.normal(0, 1, size=(batch_size, z_dim))
        embedding_batch = embeddings_test[0:batch_size]
        fake_images, _ = stage1_gen.predict_on_batch([embedding_batch, z_noise2])

        # Save images
        for i, img in enumerate(fake_images[:10]):
            save_rgb_img(img, "results/gen_{}_{}.png".format(epoch, i))

# Save models
stage1_gen.save_weights("stage1_gen.h5")
stage1_dis.save_weights("stage1_dis.h5")

g_loss:[0.32744497060775757, 0.3274223804473877, 1.1300338883302175e-05]
Batch:56
d_loss_real:0.32909756898880005
d_loss_fake:9.23323000279197e-07
d_loss_wrong:0.0011299782199785113
d_loss:0.16483150988014472
g_loss:[0.32813164591789246, 0.32811158895492554, 1.0035710147349164e-05]
Batch:57
d_loss_real:0.32939162850379944
d_loss_fake:2.7843034331453964e-06
d_loss_wrong:0.0009865396423265338
d_loss:0.16494314523833964
g_loss:[0.3278532326221466, 0.32783496379852295, 9.133633284363896e-06]
Batch:58
d_loss_real:0.32718008756637573
d_loss_fake:2.573680148998392e-07
d_loss_wrong:0.0009111245744861662
d_loss:0.16381788926881313
g_loss:[0.32753196358680725, 0.32751986384391785, 6.049717740097549e-06]
Batch:59
d_loss_real:0.3296571373939514
d_loss_fake:4.3166730279153853e-07
d_loss_wrong:0.0006688160938210785
d_loss:0.16499588063725668
g_loss:[0.3257094621658325, 0.32569849491119385, 5.4840556913404725e-06]
Batch:60
d_loss_real:0.33079713582992554
d_loss_fake:8.628777550256927e-07
d_loss_wrong

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


d_loss_real:0.327606201171875
d_loss_fake:4.0065464190774946e-07
d_loss_wrong:0.0008653430850245059
d_loss:0.1640195365208541
g_loss:[0.32651934027671814, 0.326468825340271, 2.5251372790080495e-05]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch is: 995
Number of batches 138
Batch:1
d_loss_real:0.3266987204551697
d_loss_fake:3.5824717770083225e-07
d_loss_wrong:0.0007748138741590083
d_loss:0.16354315325791902
g_loss:[0.3265138268470764, 0.3264722228050232, 2.080421836581081e-05]
Batch:2
d_loss_real:0.3270425796508789
d_loss_fake:3.301193487459386e-07
d_loss_wrong:0.0008818131173029542
d_loss:0.16374182563460238
g_loss:[0.3262961506843567, 0.32626569271087646, 1.523471928521758e-05]
Batch:3
d_loss_real:0.32824379205703735
d_loss_fake:6.673535608570091e-07
d_loss_wrong:0.0010368474759161472
d_loss:0.16438127473588793
g_loss:[0.3269684314727783, 0.32692015171051025, 2.4137158106896095e-05]
Batch:4
d_loss_real:0.3281285762786865
d_loss_fake:6.473881057900144e-07
d_loss_wrong:0.0008454917697235942
d_loss:0.1642758229288006
g_loss:[0.3265424072742462, 0.32649627327919006, 2.3072419935488142e-05]
Batch:5
d_loss_real:0.3284038007259369
d_loss_fake:1.41026859523663e-07
d_loss_wrong:0.0007700086571276188
d_loss:0.16439443778396523


...

Epoch is: 997
Number of batches 138
Batch:1
d_loss_real:0.3268938362598419
d_loss_fake:5.113444103699294e-07
d_loss_wrong:0.0008264714269898832
d_loss:0.16365366382277102
g_loss:[0.32614660263061523, 0.32613784074783325, 4.377258846943732e-06]
Batch:2
d_loss_real:0.32794123888015747
d_loss_fake:5.241588496573968e-07
d_loss_wrong:0.0008301371708512306
d_loss:0.16417828477250396
g_loss:[0.32603639364242554, 0.3260297477245331, 3.3284788969467627e-06]
Batch:3
d_loss_real:0.3285011649131775
d_loss_fake:1.1115828328911448e-06
d_loss_wrong:0.0007885830709710717
d_loss:0.16444800612003974
g_loss:[0.3272665739059448, 0.3272583484649658, 4.114311195735354e-06]
Batch:4
d_loss_real:0.328737735748291
d_loss_fake:8.95821358426474e-07
d_loss_wrong:0.0006831554346717894
d_loss:0.16453988068815306
g_loss:[0.3266415596008301, 0.3266340494155884, 3.7537620301009156e-06]
Batch:5
d_loss_real:0.32852891087532043
d_loss_fake:2.1946171102626977e-07
d_loss_wrong:0.0006369788898155093
d_loss:0.1644237550255418

Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


d_loss_real:0.32880836725234985
d_loss_fake:6.602879238926107e-07
d_loss_wrong:0.0009250493021681905
d_loss:0.16463561102369795
g_loss:[0.3261691927909851, 0.3261600434780121, 4.577137133310316e-06]


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).
Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


Epoch is: 999
Number of batches 138
Batch:1
d_loss_real:0.3291921317577362
d_loss_fake:4.1853212451314903e-07
d_loss_wrong:0.0010104019893333316
d_loss:0.16484877100923256
g_loss:[0.3268636167049408, 0.32685545086860657, 4.079411610291572e-06]
Batch:2
d_loss_real:0.3283032178878784
d_loss_fake:3.3603976135054836e-07
d_loss_wrong:0.0009522332693450153
d_loss:0.1643897512712158
g_loss:[0.3267773687839508, 0.32677194476127625, 2.714564743655501e-06]
Batch:3
d_loss_real:0.3279936909675598
d_loss_fake:6.655751576545299e-07
d_loss_wrong:0.0011146172182634473
d_loss:0.16427566618213518
g_loss:[0.3266927897930145, 0.32668495178222656, 3.924628799722996e-06]
Batch:4
d_loss_real:0.3283814787864685
d_loss_fake:9.420747915100947e-07
d_loss_wrong:0.0008555172826163471
d_loss:0.16440485423258622
g_loss:[0.3265362083911896, 0.32652777433395386, 4.219542915961938e-06]
Batch:5
d_loss_real:0.3276270627975464
d_loss_fake:1.9120948024919926e-07
d_loss_wrong:0.0007939076167531312
d_loss:0.16401205610533154

In [0]:
#copying the generator model in drive to use in stage 2
!cp /content/stage1_gen.h5 /content/drive/My\ Drive

In [0]:
!cp /content/stage1_dis.h5 /content/drive/My\ Drive

In [0]:
!cp -R /content/results /content/drive/My\ Drive